In [15]:
from langchain_community.tools import tool
from dotenv import load_dotenv
import os

load_dotenv()

True

In [22]:
# create a function that fetches the weather of a given city, and convert it into a tool
import requests
import os

@tool
def get_weather(city:str)->str:
    """Fetches the current weather of a city"""
    API_KEY = os.environ["WEATHER_API_KEY"]
    
    # url from which the weather of the city will be fetched
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={os.environ["WEATHER_API_KEY"]}&units=metric"
    
    # make a request to that url, and fetch that result
    result = requests.get(url=url).json()
    
    if result.get("cod")!=200:
        return "Weather data not available"
    
    return (
        f"Temperature : {result['main']['temp']} degree celsius",
        f"Condition : {result['weather'][0]['description']}"
    )    

In [28]:
# test the tool

get_weather.invoke({"city":"New York"})

('Temperature : -6.15 degree celsius', 'Condition : clear sky')

In [29]:
# create an llm
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.3-70b-versatile")

# bind the tool to the llm
weather_llm = llm.bind_tools([get_weather])

In [30]:
# we also need to maintain a chat history, so
from langchain_core.messages import HumanMessage

chat_history = []

query = HumanMessage("Hi! how are you?")
chat_history.append(query)

In [31]:
chat_history

[HumanMessage(content='Hi! how are you?', additional_kwargs={}, response_metadata={})]

In [33]:
# pass the message to the llm, and store the AIMessage
response_1 = weather_llm.invoke(chat_history)
chat_history.append(response_1)
chat_history

[HumanMessage(content='Hi! how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing well, thanks for asking. Is there something I can help you with, or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 222, 'total_tokens': 248, 'completion_time': 0.077112707, 'completion_tokens_details': None, 'prompt_time': 0.02142741, 'prompt_tokens_details': None, 'queue_time': 0.05856979, 'total_time': 0.098540117}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bda9c-ee25-71b1-8f29-04567e19d9e3-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 222, 'output_tokens': 26, 'total_tokens': 248})]

In [34]:
# ask the llm about the weather of a particular city
query_2 = HumanMessage("Can tell me the current weather in Kerala?")
chat_history.append(query_2)

In [35]:
response_2 = weather_llm.invoke(chat_history)
chat_history.append(response_2)
response_2

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'p3cm1m3kn', 'function': {'arguments': '{"city":"Kerala"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 266, 'total_tokens': 281, 'completion_time': 0.050990017, 'completion_tokens_details': None, 'prompt_time': 0.013511519, 'prompt_tokens_details': None, 'queue_time': 0.058281461, 'total_time': 0.064501536}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bda9e-990d-7251-97a5-033f65654d50-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Kerala'}, 'id': 'p3cm1m3kn', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 266, 'output_tokens': 15, 'total_tokens': 281})

In [36]:
response_2.tool_calls[0]

{'name': 'get_weather',
 'args': {'city': 'Kerala'},
 'id': 'p3cm1m3kn',
 'type': 'tool_call'}

In [37]:
# now get the response from the tool, and append it as a ToolMessage
tool_response = get_weather.invoke(response_2.tool_calls[0])

# and append it to chat history as well
chat_history.append(tool_response)
chat_history

[HumanMessage(content='Hi! how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm doing well, thanks for asking. Is there something I can help you with, or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 222, 'total_tokens': 248, 'completion_time': 0.077112707, 'completion_tokens_details': None, 'prompt_time': 0.02142741, 'prompt_tokens_details': None, 'queue_time': 0.05856979, 'total_time': 0.098540117}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bda9c-ee25-71b1-8f29-04567e19d9e3-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 222, 'output_tokens': 26, 'total_tokens': 248}),
 HumanMessage(content='Can tell me the current weather in Kerala?', additional_kwargs={}, response_metadata={}),
 AIMessage(content=''

In [38]:
# now finally the LLM will tell you the weather of the requested city
response_3 = weather_llm.invoke(chat_history)
print(response_3.content)

The current weather in Kerala is a clear sky with a temperature of 32.82 degree celsius.
